In [26]:
from langchain_core.tools import tool
from langchain_openai import ChatOpenAI
from langchain.agents import AgentExecutor, create_openai_functions_agent
from langchain.prompts import ChatPromptTemplate, MessagesPlaceholder
import dotenv
import os
import requests

In [29]:
BASE_URL = "https://hacker-news.firebaseio.com/v0/"

def get_data(endpoint): 
    URL = f"{BASE_URL}{endpoint}.json"
    response = requests.get(URL, timeout=10)
    return response.json()

@tool
def get_top_story_ids(limit=10):
    """
    Get the top {limit} story ids from Hacker News, 
    use get_story tool to get more data about the story id
    """
    return get_data("topstories")[:limit]

@tool
def get_story(story_id):
    """
    Get the data of any given post on Hacker News
    Data includes title, url, score, by, and time
    """
    return get_data(f"item/{story_id}")

In [30]:
top_ids = get_top_story_ids.invoke({"limit": 10})
for idx, story_id in enumerate(top_ids, start=1):
    item = get_story.invoke({"story_id": top_ids[0]})
    title = item.get("title", "<no title>")
    url = item.get("url", f"https://news.ycombinator.com/item?id={story_id}")
    score = item.get("score", 0)
    by = item.get("by", "unknown")
    print(f"{idx}. {title} ({score} points by {by})")
    print(f"{url}")

1. The Agentic AI Handbook: Production-Ready Patterns (97 points by SouravInsights)
https://www.nibzard.com/agentic-handbook
2. The Agentic AI Handbook: Production-Ready Patterns (97 points by SouravInsights)
https://www.nibzard.com/agentic-handbook
3. The Agentic AI Handbook: Production-Ready Patterns (97 points by SouravInsights)
https://www.nibzard.com/agentic-handbook
4. The Agentic AI Handbook: Production-Ready Patterns (97 points by SouravInsights)
https://www.nibzard.com/agentic-handbook
5. The Agentic AI Handbook: Production-Ready Patterns (97 points by SouravInsights)
https://www.nibzard.com/agentic-handbook
6. The Agentic AI Handbook: Production-Ready Patterns (97 points by SouravInsights)
https://www.nibzard.com/agentic-handbook
7. The Agentic AI Handbook: Production-Ready Patterns (97 points by SouravInsights)
https://www.nibzard.com/agentic-handbook
8. The Agentic AI Handbook: Production-Ready Patterns (97 points by SouravInsights)
https://www.nibzard.com/agentic-handbook


## Let's create the agent

In [31]:
dotenv.load_dotenv()
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")

llm = ChatOpenAI(api_key=OPENAI_API_KEY)

prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a helpful assistant that can answer questions about Hacker News."),
    ("human", "{input}"), 
    MessagesPlaceholder(variable_name="agent_scratchpad"),
])

tools = [get_top_story_ids, get_story]

agent = create_openai_functions_agent(llm, tools=tools, prompt=prompt)
agent_executor = AgentExecutor(
    agent=agent,
    tools=tools,
    verbose=True,
    return_intermediate_steps=True,
)

result = agent_executor.invoke({"input": "What are the top trends in the recent Hacker News posts? Give me examples and explain what these projets are."})
print(result["output"])
print(result["intermediate_steps"])



> Entering new AgentExecutor chain...

Invoking: `get_top_story_ids` with `{'limit': 5}`


[46701969, 46700594, 46665987, 46695628, 46701733]
Invoking: `get_story` with `{'story_id': 46701969}`


{'by': 'SouravInsights', 'descendants': 26, 'id': 46701969, 'kids': [46703091, 46702578, 46702811, 46703147, 46702663, 46703148, 46702983, 46702360, 46702807, 46702445, 46702328], 'score': 97, 'time': 1768978136, 'title': 'The Agentic AI Handbook: Production-Ready Patterns', 'type': 'story', 'url': 'https://www.nibzard.com/agentic-handbook'}
Invoking: `get_story` with `{'story_id': 46700594}`
responded: One of the top trends on Hacker News is "The Agentic AI Handbook: Production-Ready Patterns". This project aims to provide production-ready patterns for Agentic AI. You can find more information about this project on [Nibzard's website](https://www.nibzard.com/agentic-handbook).

Let me fetch more details about the other trending posts.

{'by': 'myahio', 'descendants': 147, 'id': 46700594, 'k